In [2]:
import sys
import os
import time
import numpy as np
import tensorflow as tf
import gpflow
import soundfile
import pickle
sys.path.append('../../../../')
import gpitch
from gpitch.amtgp import logistic
from gpitch import myplots
import soundfile
import peakutils
from scipy import signal
from scipy.fftpack import fft, ifft, ifftshift
from scipy import signal

In [3]:
visible_device = sys.argv[1]  # configure gpu usage
gpitch.amtgp.init_settings(visible_device=visible_device, interactive=False)

In [6]:
pickleloc = '../../../../../results/files/svi/script/'  # location saved models
filename = '../../../../../datasets/maps/test_data/segment4-down.wav'  # loc test data

bounds = [21, 109]  # pitches to detect
midi = np.asarray([str(i) for i in range(bounds[0], bounds[1])]).reshape(-1,)  # midi list
Np = midi.size
fs = 16e3

In [7]:
m = [pickle.load(open(pickleloc +"maps_pitch_"+ midi[i] + ".p", "rb")) for i in range(Np)]
m_bg = pickle.load(open(pickleloc + "maps_background.p", "rb")) # load background model

In [8]:
y, fs = soundfile.read(filename)
y = y.reshape(-1, 1)
Ntest = y.size
x = np.linspace(0, (Ntest-1.)/fs, Ntest).reshape(-1, 1)

In [9]:
dec = 160  # decimation level
maxiter = 2000  # max number of iterations
mbs = 200  # mini batch size
learning_rate = 0.01  # learning rate
z = x[::dec].copy()  # inducing points

In [10]:
all_mean_f = [None]*Np  # lists to save results prediction for the 88 notes
all_mean_g = [None]*Np
all_var_f = [None]*Np
all_var_g = [None]*Np

In [11]:
Nh = 20  # max number of harmonic for the component kernels
ker_com_pitch = gpitch.kernels.MaternSpecMix(Nc=Nh)  # init kernels
ker_act_pitch = gpflow.kernels.Matern32(input_dim=1, lengthscales=0.1, variance=10.)
ker_com_bg = m_bg.kern_com
ker_act_bg = m_bg.kern_act
kc = [ker_com_pitch, ker_com_bg]
ka = [ker_act_pitch, ker_act_bg]

In [12]:
mloo = gpitch.loogp.LooGP(X=x, Y=y, kf=kc, kg=ka, Z=z, minibatch_size=mbs)  # init model

In [24]:
mloo.kern_f1.fixed = True
mloo.kern_f2.fixed = True
mloo.kern_g1.fixed = False
mloo.kern_g2.fixed = False
mloo.likelihood.variance.fixed = False